# SMU-MSDS-7333-Quantifying the World

## Case Study 3- Email Classification as Span or Ham 

Team Members:
* Rashmi Patel
* Simarpreet Reddy


# Introduction

In this case study, we will build a classifier using clustering and naive bayes classifier that classifies whether the email given is spam or ham. We will perform analysis by dividing this case study into 6 parts:

* Business Understanding

* Data Evaluation / Engineering

* Modeling Preparations

* Model Building & Evaluation

* Model Interpretability & Explainability

* Case Conclusions

In [5]:
#Import required libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import sklearn as sk
import re
import os

#NLP
import nltk
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

#Email Parser 
import email
from html.parser import HTMLParser
from bs4 import BeautifulSoup

## Business Understanding

## Data Evaluation/ Engineering

In [7]:
os.listdir()

['spam',
 'hard_ham',
 'spam_2',
 '.DS_Store',
 'easy_ham',
 'RP_SR_7333_CaseStudy3.ipynb',
 '.ipynb_checkpoints',
 'easy_ham_2']

In [24]:
root = '/Users/arth/Desktop/RP-MSDS/Spring2022/QTW/CaseStudy3'
for item in os.listdir(root):
    if not item.startswith('.') and os.path.isfile(os.path.join(root, item)):
        print(item)

RP_SR_7333_CaseStudy3.ipynb


In [25]:
folder = [fdr for fdr in os.listdir('.') if (re.match(r'easy', fdr)) or (re.match(r'hard', fdr)) or (re.match(r'spam', fdr))]
folder

['spam', 'hard_ham', 'spam_2', 'easy_ham', 'easy_ham_2']

In [30]:
directory = []
for i in range(len(folder)):
    directory = folder[i]
    print(directory,':',len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))]))


spam : 1001
hard_ham : 501
spam_2 : 934
easy_ham : 5052
easy_ham_2 : 1401


In [70]:
# for SAPM MAILS

spam_root = '/Users/arth/Desktop/RP-MSDS/Spring2022/QTW/CaseStudy3/spam/'

spam_files=[name for name in os.listdir(folder[0]) if os.path.isfile(os.path.join(folder[0], name))]
#print(spam_files)

for file in spam_files:
            with open(spam_root+ file, encoding='latin1') as f:
                lines = f.readlines()
                f.close()
            
            with open(spam_root+ file, encoding='latin1') as f:
                body = f.read()
                f.close()
                
       

In [71]:
msg = email.message_from_string(str(body))
#print(msg)
            

Return-Path: <aileen@email2.qves.net>
Delivered-To: zzzz@localhost.spamassassin.taint.org
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 9EA0044157
	for <zzzz@localhost>; Fri, 23 Aug 2002 06:02:51 -0400 (EDT)
Received: from mail.webnote.net [193.120.211.219]
	by localhost with POP3 (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Fri, 23 Aug 2002 11:02:51 +0100 (IST)
Received: from email.qves.com ([67.104.83.251])
	by webnote.net (8.9.3/8.9.3) with ESMTP id WAA06881
	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 22:34:30 +0100
Received: from qvp0087 ([169.254.6.20]) by email.qves.com with Microsoft SMTPSVC(5.0.2195.2966);
	 Thu, 22 Aug 2002 15:09:13 -0600
From: "FreeLegal Advice" <aileen@email2.qves.net>
To: <zzzz@spamassassin.taint.org>
Subject: Get the Child Support You Deserve                            11.180
Date: Thu, 22 Aug 2002 15:09:11 -0600
Message-ID: <110d1201c24a20$29b657e0$1406fea9@freeyankeedom.co

In [74]:
tmpStr = ''
            
if msg.is_multipart():
    for payload in msg.get_payload():
        tmpStr = ' '.join(str(payload.get_payload()))
else:
    bodies.append(str(msg.get_payload()))
    

NameError: name 'bodies' is not defined

In [81]:
if msg.get_payload():
    msg.get_payload()

In [ ]:
targets = []
data = []
count = 0
not_spamList = os.listdir("./easy_ham/")
for i in not_spamList:
    with open("./easy_ham/"+i, "r",encoding= 'utf-8', errors='ignore') as f:
        x = email.message_from_file(f)
    x = return_text(x)
    mailType = x.get_content_type()
    if mailType == "text/plain":
        tmp = x.get_payload()
        tmp = tmp.replace("\n"," ")
        tmp = tmp.replace(r'http\S+', ' ')
        tmp = tmp.replace("-"," ")
        data.append(tmp)
        targets.append(0)
    elif mailType == "text/html":
        tmp = BeautifulSoup(x.get_payload(), 'html.parser')
        tmp = tmp.text.replace("\n"," ")
        tmp = tmp.replace(r'http\S+', ' ')
        tmp = tmp.replace("-"," ")
        data.append(tmp)
        targets.append(0)
    else:
        print(f"msg {i} not parsed")
        print(x.get_content_type())
print('Number of emails parsed in easy_ham:',len(data))
print('Number of targets for easy_ham:',len(targets))

In [ ]:
def return_text(mail):
    if mail.is_multipart():
        tmp = mail.get_payload()[0]
        return return_text(tmp)
    elif mail.is_multipart() == False:
        return mail

In [99]:
# for SAPM MAILS

spam_root = '/Users/arth/Desktop/RP-MSDS/Spring2022/QTW/CaseStudy3/spam/'

spam_files=[name for name in os.listdir(folder[0]) if os.path.isfile(os.path.join(folder[0], name))]
#print(spam_files)

for file in spam_files:
    with open(spam_root+ file,'r', encoding='latin1') as f:
        tmp= email.message_from_file(f)
    if tmp.is_multipart():
        x=tmp.get_payload()[0]
    elif tmp.is_multipart()==False:
        t= tmp
print(x) 
print('=======================')
print(t)

Content-Type: text/plain;
	charset="Windows-1252"
Content-Transfer-Encoding: 7bit

 PruLife Universal Protector
 - and -
 Prulife Universal Plus

  ROCK!
  

Case Closed! Permanent Life Insurance at Affordable Rates

Whether your clients are looking to pay the Minimum Premiums needed to
Endow* a policy at a specific age or pay sufficient premiums needed to
guarantee** a death benefit for
life, Prudential has the answer.

PruLife Universal ProtectorSM offers competitive lifetime death benefit
guarantee premiums. PruLife Universal PlusSM offers competitive minimum
premiums to endow and competitive long term cash value accumulation.

And that's not all. Look what else Prudential+ has to offer...

  _____  

COMPETITIVE COMPENSATION	 
 	 Rolling Commissionable Target Premiums (R-CTP)++.
First year compensation will be paid until the R-CTP is reached during
the first 24 policy months. 	
 	 First year commissions that do not reduce at any issue age. 	
 COMPETITIVE UNDERWRITING	 
 	 Issue age

In [102]:
import chardet

# for SAPM MAILS

spam_root = '/Users/arth/Desktop/RP-MSDS/Spring2022/QTW/CaseStudy3/spam/'

spam_files=[name for name in os.listdir(folder[0]) if os.path.isfile(os.path.join(folder[0], name))]
#print(spam_files)

for file in spam_files:
    with open(spam_root+ file,'r', encoding='latin1') as f:
        tmp= email.message_from_file(f)
        tmp=get_text(tmp)



def get_text(msg):
    """ Parses email message text, given message object
    This doesn't support infinite recursive parts, but mail is usually not so naughty.
    """
    text = ""
    if msg.is_multipart():
        html = None
        for part in msg.get_payload():
            if part.get_content_charset() is None:
                charset = chardet.detect(str(part))['encoding']
            else:
                charset = part.get_content_charset()
            if part.get_content_type() == 'text/plain':
                text = unicode(part.get_payload(decode=True),str(charset),"ignore").encode('utf8','replace')
            if part.get_content_type() == 'text/html':
                html = unicode(part.get_payload(decode=True),str(charset),"ignore").encode('utf8','replace')
            if part.get_content_type() == 'multipart/alternative':
                for subpart in part.get_payload():
                    if subpart.get_content_charset() is None:
                        charset = chardet.detect(str(subpart))['encoding']
                    else:
                        charset = subpart.get_content_charset()
                    if subpart.get_content_type() == 'text/plain':
                        text = unicode(subpart.get_payload(decode=True),str(charset),"ignore").encode('utf8','replace')
                    if subpart.get_content_type() == 'text/html':
                        html = unicode(subpart.get_payload(decode=True),str(charset),"ignore").encode('utf8','replace')

        if html is None:
            return text.strip()
        else:
            return html.strip()
    else:
        text = unicode(msg.get_payload(decode=True),msg.get_content_charset(),'ignore').encode('utf8','replace')
        return text.strip()
    
    
print(msg)


NameError: name 'unicode' is not defined

## Modeling Preparations

## Model building & Evaluations

## Model Interpretabilty & EDxplainability

## Case Conclusions